In [65]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd # geospatial data processing
import numpy as np # linear algebra operations
import matplotlib.pyplot as plt # plotting
import sklearn.linear_model as lm # linear regression
from sklearn.feature_selection import SequentialFeatureSelector # subset selection
from sklearn.model_selection import train_test_split # train-test split of data
from sklearn.metrics import mean_squared_error # MSE calculator

# Import and Clean Data

### Importing the Data

In [66]:
housing_org = pd.read_csv("housing.csv", low_memory=False)  # read in data
crimes_org = pd.read_csv("crimes.csv", low_memory=False)
parks_org = gpd.read_file("park.geojson")
tract_boundaries = gpd.read_file('b.geojson')\
# shoudl take around 30-40 seconds to run

### Filtering the Data

In [67]:
crimes_filt = crimes_org[crimes_org['Latitude'] > 41 ]
crimes_filt = crimes_org[crimes_org['Latitude'] < 43]
crimes_filt = crimes_org[crimes_org['Longitude'] > -88]
crimes_filt = crimes_org[crimes_org['Longitude'] < -87.5]

parks_org.x_coord = parks_org.x_coord.astype(float)
parks_org.y_coord = parks_org.y_coord.astype(float)
parks_org.objectid_1 = parks_org.objectid_1.astype(int)
parks_org.gisobjid = parks_org.gisobjid.astype(float)
parks_org.park_no = parks_org.park_no.astype(float)

parks_filt = parks_org[(parks_org['y_coord']) > 41 ]
parks_filt = parks_org[parks_org['y_coord'] < 43]
parks_filt = parks_org[parks_org['x_coord'] > -88]
parks_filt = parks_org[parks_org['x_coord'] < -87.5]

In [68]:
housing_filt = housing_org[
    [
        "Property Index Number",
        "Property Class",
        "Township Name",
        "Number of Active 288s (HIEs)",
        "Number of Ended 288s (HIEs)",
        "Sale Price",
        "Land Square Feet",
        "Large Home Indicator",
        "Age",
        "Apartments",
        "Wall Material",
        "Roof Material",
        "Rooms",
        "Bedrooms",
        "Basement",
        "Basement Finish",
        "Central Heating",
        "Central Air Conditioning",
        "Fireplaces",
        "Attic Type",
        "Full Baths",
        "Half Baths",
        "Cathedral Ceiling",
        "Garage 1 Size",
        "Garage 1 Material",
        "Garage 1 Attached",
        "Garage 1 Area",
        "Building Square Feet",
        "Repair Condition",
        "Attic Finish",
        "Renovation",
        "Porch",
        "Tract GEOID",
        "O'Hare Noise Indicator",
        "FEMA Floodplain",
        "Flood Risk Factor",
        "Flood Risk Direction",
        "Road Proximity < 100 Feet",
        "Road Proximity 101 - 300 Feet",
        "Elementary/Middle School District",
        "High School District",
        "Longitude",
        "Latitude",
        "Tax Rate",
        "Tract Median Income",
        "Modeled Sale Year",
        "Modeled Sale Month",
        "Modeled Sale Day",
        "Modeled Sale During School Year",
        "Modeled Sale During Holidays",
    ]
]
housing_filt.dropna(inplace=True) # drop rows with missing values

/var/folders/w5/_4fbzd_52lz_ghtnf_r3jf7c0000gn/T/ipykernel_2752/1241320180.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_filt.dropna(inplace=True) # drop rows with missing values


### Cleaning Data

In [69]:
if 'Property Index Number' in housing_filt.columns: # set PIN as index
	housing_filt.set_index('Property Index Number', inplace=True)

for x in housing_filt.columns: # convert all columns to numeric 
	if housing_filt[x].dtype == 'object':
		housing_filt[x].replace(housing_filt[x].unique(), range(len(housing_filt[x].unique())), inplace=True)
		housing_filt[x] = housing_filt[x].astype('category')

/var/folders/w5/_4fbzd_52lz_ghtnf_r3jf7c0000gn/T/ipykernel_2752/4270220874.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_filt[x].replace(housing_filt[x].unique(), range(len(housing_filt[x].unique())), inplace=True)
/var/folders/w5/_4fbzd_52lz_ghtnf_r3jf7c0000gn/T/ipykernel_2752/4270220874.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housing_filt[x] = housing_filt[x].astype('category')
/var/folders/w5/_4fbzd_52lz_ghtnf_r3jf7c0000gn/T/ipykernel_2752/4270220874.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

### Constructing Geospatial DataSets

In [70]:
# tract_boundaries.geoid10 = tract_boundaries.geoid10.astype(int)

housing = gpd.GeoDataFrame(housing_filt, crs="EPSG:4326", geometry=gpd.points_from_xy(housing_filt.Longitude, housing_filt.Latitude))
housing.rename(columns={'Tract GEOID': 'geoid10'}, inplace=True)

crimes = gpd.GeoDataFrame(
    crimes_filt,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(crimes_filt. Longitude, crimes_filt.Latitude),
)
crimes = gpd.sjoin(crimes, tract_boundaries, how="left", predicate="within")

parks = gpd.GeoDataFrame(
    parks_filt,
    crs="EPSG:4326",
    geometry=gpd.points_from_xy(parks_filt.x_coord, parks_filt.y_coord),
)
parks = gpd.sjoin(parks, tract_boundaries, how="left", predicate="within")

In [74]:

print(crimes.geoid10.unique())
print(housing.geoid10.unique())


['17031610300' '17031251200' '17031210601' '17031251800' '17031710700'
 '17031831000' '17031840700' '17031837100' '17031650500' '17031110501'
 '17031490700' '17031390100' '17031241400' '17031710600' '17031251600'
 '17031510200' '17031711300' '17031150600' '17031520200' '17031120400'
 '17031430101' '17031431400' '17031431302' '17031460302' '17031431301'
 '17031839100' '17031834600' '17031242100' '17031837000' '17031140500'
 '17031292400' '17031020801' '17031690300' '17031230600' '17031680900'
 '17031680600' '17031351400' '17031252102' '17031081600' '17031681000'
 '17031837300' '17031691000' '17031837400' '17031031700' '17031060900'
 '17031440900' '17031140302' '17031080202' '17031231200' '17031282700'
 '17031530501' '17031500300' '17031490901' '17031460301' '17031330100'
 '17031130300' '17031491300' '17031351000' '17031431200' '17031530100'
 '17031834700' '17031842200' '17031740200' '17031191301' '17031490902'
 '17031833000' '17031611100' '17031611800' '17031243400' '17031020602'
 '1703

In [89]:
crime_counts = crimes.groupby("geoid10", as_index=False)["ID"].count()
park_counts = parks.groupby("geoid10", as_index=False)["objectid_1"].count()
housing_avgs = (
    housing.groupby("geoid10", as_index=False)["Sale Price"].mean().reset_index()
)

housing_avgs.geoid10 = housing_avgs.geoid10.astype(int)

districts = crime_counts.merge(park_counts, on="geoid10")
districts.geoid10 = districts.geoid10.astype(int)
districts = districts.merge(housing_avgs, on="geoid10")
districts.rename(columns={"Sale Price": "avg_sale"}, inplace=True)
districts.rename(columns={"ID": "crime_count"}, inplace=True)
districts.rename(columns={"objectid_1": "park_count"}, inplace=True)

In [90]:
print(districts.head())

       geoid10  crime_count  park_count  index  avg_sale
0  17031140500         5851           1      1  205000.0
1  17031220702         8274          15      4  525000.0
2  17031240900        10406          57      6  505000.0
3  17031290900        22672          22      8   51500.0
4  17031301600         7060           1      9  145000.0
